<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 07**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**

*   Iniciar o estudo do analisador semântico
*   Entender o processo de decoração da árvore sintática
*   Aplicar o design pattern Visitor para realizar o processo de decoração da árvore sintática


In [ ]:
!pip install rply

**ANALISADOR SEMÂNTICO**

O análise semântica é o terceiro passo no processo de compilação e é realizada pelo **analisador semântico**. 

<img src="https://www.researchgate.net/profile/Nuno-Oliveira-15/publication/266497079/figure/fig1/AS:295651775664128@1447500284302/Common-Compiler-Phases.png"> <img>

O analisador semântico realiza diversas operações como, por exemplo, decoração da árvore sintática, verificação de tipos e verificação de chamadas de procedimentos e funções.



**DECORAÇÃO DA ÁRVORE SINTÁTICA**

O objetivo da decoração da árvore sintática consiste em adicionar informações semânticas na árvore sintática (tipo, quantidade de parâmetros, contexto, etc) de forma a facilitar os passos seguintes da análise semântica.

Por exemplo, para o processo de verificação de tipos, precisamos ter todos os nós da árvore sintática (relacionados com tipos) com os seu respectivos tipos decorados na árvore. Na chamada de um procedimento e função, precisamos saber se estamos usando o número correto de parâmetros e estamos usando seus tipos corretamente.

Abaixo, temos um exemplo de erro semântico:

**int x=1,y;**

**string nome="maria";**

**y=x+nome;**

Observe que não há erro sintático. Porém, ao tentarmos somar x(inteiro) com nome(string), teremos uma incompatibilidade de tipos na soma. Para tratar isto, fazemos o processo em três passos:

(1) montamos uma tabela de símbolos com a declaração das variáveis:

<table>
<tr> <td> <b>id</b> </td> <td> <b>type</b> </td> </tr>
<tr> <td> x </td> <td> int </td> </tr>
<tr> <td> y </td> <td> int </td> </tr>
<tr> <td> nome </td>  <td> string </td>  </tr> 
</table>

(2) decoramos os nós da árvore relativos à expressão aritmética:

**y<font color="red">(int)</font>=x<font color="red">(int)</font>+nome<font color="red">(string)</font>;**

(3) no operador de soma, verificamor que os operandos x<font color="red">(int)</font> e nome<font color="red">(string)</font> são incompatíveis.

**Cada um destes passos é realizado por um Visitor.**


**DESIGN PATTERN VISITOR**

O Visitor é um design pattern comportamental que permite adicionar novos comportamentos à hierarquia de classes existente sem alterar nenhum código existente.

<img src=https://www.oodesign.com/images/behavioral/visitor-pattern.png> </img>


Vamos implementar este design pattern para as classes abaixo:

In [ ]:
class Expr(object):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)


class Int(Expr):
    def __init__(self, value):
        self.value = value


class Add(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Mul(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right


In [ ]:
class Visitor(object):
    pass


class Eval(Visitor):
    def visit_int(self, i):
        return i.value

    def visit_add(self, a):
        return a.left.accept(self) + a.right.accept(self)

    def visit_mul(self, a):
        return a.left.accept(self) * a.right.accept(self)


class Print(Visitor):
    def visit_int(self, i):
        return i.value

    def visit_add(self, a):
        return '(+ {} {})'.format(a.left.accept(self), a.right.accept(self))

    def visit_mul(self, a):
        return '(* {} {})'.format(a.left.accept(self), a.right.accept(self))


In [ ]:
expr = Add(Add(Int(4), Int(3)), Mul(Int(10), Add(Int(1), Int(1))))
print(expr.accept(Print()))
print(expr.accept(Eval()))

(+ (+ 4 3) (* 10 (+ 1 1)))
27


**EXERCÍCIOS**

Implementar um visitor de impressão e outro de cálculo da expressão para a gramática abaixo:

\<expression\> ::= NUMBER

       | \<expression\> "+" \<expression\>
 
       | \<expression\> "-" \<expression\>
 
       | \<expression\> "*" \<expression\>
 
       | \<expression\> "/" \<expression\>
 
       | "(" <expression> ")"

In [ ]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

In [ ]:
from rply.token import BaseBox

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass
    

class Sub(BinaryOp):
  pass
   

class Mul(BinaryOp):
  pass
    

class Div(BinaryOp):
  pass
    


In [ ]:
class Visitor(object):
    pass


class Eval(Visitor):
    def visit_number(self, i):
        return i.value
        
    def visit_add(self, a):
        return a.left.accept(self) + a.right.accept(self)

class Print(Visitor):
    def visit_number(self, i):
        return i.value

In [ ]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

@pg.production('expression : NUMBER')
def expression_number(p):
    # p is a list of the pieces matched by the right hand side of the
    # rule
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [ ]:
arvore=parser.parse(lexer.lex('1+2'))
#print(arvore.accept(Print()))
print(arvore.accept(Eval()))

3
